In [1]:
import gc
import torch
from typing import List, TypedDict
from pprint import pprint

from pydantic import BaseModel, Field
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_tavily import TavilySearch
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

In [14]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})
llm_eval = ChatOpenAI(model='gpt-5-nano', temperature=0)
llm_gen = ChatOpenAI(model='gpt-5-mini', temperature=0.1)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [ ]:
urls = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/javaguide.html",
    "https://google.github.io/styleguide/jsguide.html"
]

headers_to_split_on = [
    ('h1', 'Header 1'),
    ('h2', 'Header 2'),
    ('h3', 'Header 3')
]
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = []
for url in urls:
    splits = html_splitter.split_text_from_url(url)
    html_header_splits.extend(splits)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=50)
doc_splits = text_splitter.split_documents(html_header_splits)

In [7]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name='rag-chroma-2',
    embedding=embeddings
)

retriever = vectorstore.as_retriever()

In [8]:
for chunk in doc_splits[:3]: # 앞부분 3개만 확인
    print(f"Content Preview: {chunk.page_content[:50]}...")
    print(f"Metadata: {chunk.metadata}") 
    print("-" * 20)

Content Preview: AUTHORS:
Prefer only GitHub-flavored Markdown in e...
Metadata: {}
--------------------
Content Preview: Google Python Style Guide...
Metadata: {'Header 1': 'Google Python Style Guide'}
--------------------
Content Preview: Table of Contents  
1 Background  
2 Python Langua...
Metadata: {'Header 1': 'Google Python Style Guide'}
--------------------


In [ ]:
class GradeDocuments(BaseModel):
    binary_score: str = Field(description='문서와 질문의 연관성 여부. (yes or no)')

structured_llm_grader = llm_eval.with_structured_output(GradeDocuments)
system = '''
당신은 사용자의 질문에 대해 검색된 문서의 관련성을 평가하는 전문가입니다.
문서에 질문과 관련된 키워드나 의미가 담겨 있으면, 해당 문서를 "관련 있음"으로 평가하세요.
문서가 질문과 관련이 있는지 여부를 "yes" 또는 "no"로 표시해주세요.'''
grade_prompt = ChatPromptTemplate.from_messages(
    [('system', system),
     ('human', '검색된 문서: \n\n {document} \n\n사용자 질문: {question}')]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [13]:
question = '파이썬 코드 작성 가이드'

test = retriever.invoke(question)
test_txt = test[0].page_content
print(retrieval_grader.invoke({'question':question, 'document':test_txt}))

binary_score='yes'


In [16]:
system = '''
당신은 질문에 답변하는 업무를 돕는 도우미입니다.
제공된 문맥을 바탕으로 질문에 답변하세요. 만약 답을 모른다면 모른다고 말하세요.
세 문장을 넘지 않도록 답변을 간결하게 작성하세요.
'''
gen_prompt = ChatPromptTemplate.from_messages(
    [('system', system),
     ('human', '질문: {question} \n문맥: {context} \n답변:')]
)

rag_chain = gen_prompt | llm_gen | StrOutputParser()

In [17]:
generation = rag_chain.invoke(
    {'context':format_docs(doc_splits), 'question':question}
)
print(generation)

요약 가이드: pylint로 린트하고(부적절한 경고는 이유를 붙여 좁은 범위로 억제), 기본 라인 길이는 80자(예외: 긴 import/URL 등), 들여쓰기는 4칸, import는 모듈/패키지 전체 경로로(상대 import 금지), 네이밍은 함수/변수 snake_case, 클래스 UpperCamelCase, 상수 ALL_CAPS.  
문서화/문자열/로깅: 공개 API엔 3중 따옴표 docstring(요약문 + 필요시 Args/Returns/Raises), 문자열은 f-string/format/% 사용, 로깅은 패턴 문자열 리터널과 인자를 따로 전달(예: logger.info('msg %s', val)).  
기타 중요 규칙: 가변 기본값 금지(대신 None 처리), 파일/소켓 등은 with로 명시적 종료, 구체적 예외 사용·범용 except 금지·try/except 범위 최소화, 프로퍼티·데코레이터·고급 기능은 꼭 필요할 때만, 타입 애노테이션(pytype 등) 권장.


In [23]:
system = '''
당신은 입력된 질문을 변형하여 웹 검색에 최적화된 형태로 만드는 질문 생성기입니다.
입력된 질문을 보고 그 이면에 있는 의미나 의도를 파악해주세요.
'''
rewrite_prompt = ChatPromptTemplate.from_messages(
    [('system', system),
     ('human', '질문: \n\n {question} \n더 나은 질문으로 바꿔주세요. 단 최적의 질문 하나만 골라서 출력해주세요.')]
)

question_rewriter = rewrite_prompt | llm_gen | StrOutputParser()

In [24]:
question = 'C++ 깔끔하게 짜고 싶다.'
question_rewriter.invoke({'question':question})

'C++ 코드를 더 깔끔하고 유지보수하기 쉽게 작성하려면 어떤 모범 사례(스타일 가이드, 모던 C++ 기법, 디자인 원칙)와 도구(예: clang-format, clang-tidy), 리팩토링 기법을 사용해야 하나요?'

In [20]:
search_tool = TavilySearch(k=3)